# Setup

In [1]:
import pyphi
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import patsy
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import binarize

In [2]:
%matplotlib inline

In [3]:
import pixiedust

Pixiedust database opened successfully


## Immutables

In [4]:
RAW_DATA_DIR = "../data/raw/"
SAMPLE_DATA_FILE = "split2250_bipolarRerefType1_lineNoiseRemoved_postPuffpreStim.mat"

In [5]:
FLY_DATA = sio.loadmat(RAW_DATA_DIR + SAMPLE_DATA_FILE).get("fly_data")

# Processing Functions

In [6]:
def gen_log_reg(data):
    """ Generate logistic regression for binarised past and present states.
    
    Args:
        data (array): (timepoints, channels, trials) array of binarised data.
                      The data will pool each timepoint step as a separate trial.
                      
    Returns: 
        List of Logistic regressions (fitted) for each channel.
    """
    
    n_timepoints, n_channels, n_trials = data.shape
    
    samples = np.zeros((n_channels * 2, (n_trials * (n_timepoints - 1))))
    
    for i_p in range(n_timepoints - 1):
        sliced = data[i_p:i_p+2, :, :]
        new_sample = sliced.reshape((n_channels * 2, n_trials))
        samples[:, i_p * n_trials:(i_p+1)*n_trials] = new_sample
    
    #X = samples[0:n_channels, :].transpose()
    
    # TODO find faster way to construct dict
    data_dict = {"x{}".format(i_x): samples[i_x] for \
                i_x in range(n_channels)}
    
    patsy_str = "*".join(["x{}".format(i_x) for i_x in range(n_channels)])
    
    X_dmatrix = patsy.dmatrix(patsy_str, data_dict)
    Xs = X_dmatrix[:, 1:] # exclude intercept term
    
    models = []
    
    for i_c in range(n_channels):
        y = samples[n_channels + i_c]
        lr = LogisticRegression(solver='lbfgs')
        model = lr.fit(Xs, y)
        models.append(model)
    
    return models

In [7]:
def models_to_tpm(models, n_channels):
    """ Converts a logistic regression model to a TPM
    
    Args:
        models: List of fitted logistic regression models.
        n_channels (int): Number of channels used to generate model.
        
    Returns:
        A numpy array as a TPM.
    """
    
    tpm_shape = [2] * n_channels + [n_channels]
    
    tpm = np.zeros(tpm_shape)
    
    for state in itertools.product((0, 1), repeat=n_channels):
        for i_m, model in enumerate(models):
            state_arr = np.array(state)
            data_dict = {"x{}".format(i_x): state_arr[i_x] for \
                        i_x in range(n_channels)}
            patsy_str = "*".join(["x{}".format(i_x) for i_x in range(n_channels)])
            state_dmatrix = patsy.dmatrix(patsy_str, data_dict)
            state_interact = state_dmatrix[:, 1:]
            tpm[state + (i_m,)] = model.predict_proba(state_interact)[0][1]
    
    return tpm

In [8]:
def tpm_log_reg(data):
    """ Generate tpm using log regression for binarised past and present states.
    
    Args:
        data (array): (timepoints, channels, trials) array of binarised data.
                      The data will pool each timepoint step as a separate trial.
                      
    Returns: 
        TPM for the input data.
    """
    
    _, n_channels, _ = data.shape
    
    return models_to_tpm(gen_log_reg(data), n_channels)

# Testing

## Generated Data (Deterministic)

In [9]:
def det_generate(tpm, n_timepoints, n_channels, n_trials):
    
    states = list(itertools.product((0, 1), repeat=n_channels))
    
    samples = np.zeros((1 + n_timepoints, n_channels, (2 ** n_channels) * n_trials))
    
    for i_s, state in enumerate(states * n_trials):
        samples[0, :, i_s] = np.array(state)
        
        for i_t in range(n_timepoints):
            curr_state = tuple(samples[i_t, :, i_s].astype(int))
            value = tpm[curr_state]
            samples[i_t + 1, :, i_s] = value
    
    return samples

In [17]:
test_shape = [2] * 3 + [3]
test_tpm = np.zeros(test_shape)

test_tpm[(0, 0, 0)] = np.array([0, 0, 0])
test_tpm[(1, 0, 0)] = np.array([0, 0, 1])
test_tpm[(0, 1, 0)] = np.array([1, 0, 1])
test_tpm[(1, 1, 0)] = np.array([1, 0, 0])
test_tpm[(0, 0, 1)] = np.array([1, 1, 0])
test_tpm[(1, 0, 1)] = np.array([1, 1, 1])
test_tpm[(0, 1, 1)] = np.array([1, 1, 1])
test_tpm[(1, 1, 1)] = np.array([1, 1, 0])

### 10 Trials

In [18]:
test_samples_10_trials = det_generate(test_tpm, 100, 3, 10)

In [19]:
out_tpm_10_trials = tpm_log_reg(test_samples_10_trials).round(decimals=1)

In [20]:
for state in itertools.product((0, 1), repeat=3):
    print("STATE = {}, IN_TPM = {}, OUT_TPM = {}".format(state, 
                                                         test_tpm[state], 
                                                         out_tpm_10_trials[state]))

STATE = (0, 0, 0), IN_TPM = [0. 0. 0.], OUT_TPM = [0. 0. 0.]
STATE = (0, 0, 1), IN_TPM = [1. 1. 0.], OUT_TPM = [1. 1. 0.]
STATE = (0, 1, 0), IN_TPM = [1. 0. 1.], OUT_TPM = [0.9 0.  0.1]
STATE = (0, 1, 1), IN_TPM = [1. 1. 1.], OUT_TPM = [1.  1.  0.5]
STATE = (1, 0, 0), IN_TPM = [0. 0. 1.], OUT_TPM = [0. 0. 1.]
STATE = (1, 0, 1), IN_TPM = [1. 1. 1.], OUT_TPM = [1. 1. 1.]
STATE = (1, 1, 0), IN_TPM = [1. 0. 0.], OUT_TPM = [1. 0. 0.]
STATE = (1, 1, 1), IN_TPM = [1. 1. 0.], OUT_TPM = [1. 1. 0.]


### 50 Trials

In [21]:
test_samples_50_trials = det_generate(test_tpm, 100, 3, 50)

In [22]:
out_tpm_50_trials = tpm_log_reg(test_samples_50_trials).round(decimals=1)

In [23]:
for state in itertools.product((0, 1), repeat=3):
    print("STATE = {}, IN_TPM = {}, OUT_TPM = {}".format(state, 
                                                         test_tpm[state], 
                                                         out_tpm_50_trials[state]))

STATE = (0, 0, 0), IN_TPM = [0. 0. 0.], OUT_TPM = [0. 0. 0.]
STATE = (0, 0, 1), IN_TPM = [1. 1. 0.], OUT_TPM = [1. 1. 0.]
STATE = (0, 1, 0), IN_TPM = [1. 0. 1.], OUT_TPM = [0.9 0.  0.6]
STATE = (0, 1, 1), IN_TPM = [1. 1. 1.], OUT_TPM = [1.  1.  0.9]
STATE = (1, 0, 0), IN_TPM = [0. 0. 1.], OUT_TPM = [0. 0. 1.]
STATE = (1, 0, 1), IN_TPM = [1. 1. 1.], OUT_TPM = [1. 1. 1.]
STATE = (1, 1, 0), IN_TPM = [1. 0. 0.], OUT_TPM = [1. 0. 0.]
STATE = (1, 1, 1), IN_TPM = [1. 1. 0.], OUT_TPM = [1. 1. 0.]


> As a note, it seems when the number of trials is low and a certain state does not occur frequently, the transition probability from that state is quite poorly estimated (as can be seen from the comparison from 10 trials and 50 trials above). 

## Fly Data

In [ ]:
FLY_DATA.shape

The shape of the data corresponds to:

- 2350 time samples (at 1000Hz)
- 15 channels (bipolar re-referenced)
- 8 "trials" (from one trial cut into smaller trials)
- 13 flies
- 2 conditions (awake and anaesthetised respectively)

The reference paper is at https://doi.org/10.1523/ENEURO.0329-17.2018